In [2]:
import os
import re
import pandas as pd
import numpy as np
from src.tools.geomap_tools import haversine

In [3]:
base_folder = "data/external/adsb/"
year_selected = "2024"
sites = pd.read_csv("data/external/sites.csv")
sites.loc[sites.name=="EIH", "name"] = "BRE"
sites.rename(columns={"id" : "site_id", "name" : "site_abrev_name", "basename" : "site_name", "code" : "site_code"}, inplace=True)
time_threshold = pd.Timedelta(minutes=20)
maximum_altitude = 8000 # feet
maximum_distance = 20 # km

In [4]:
final_table = []

# Traverse through all folders and subfolders
for root, dirs, files in os.walk(base_folder):
    folder_dfs = []
    # Iterate through files in the current folder
    for file in files:

        if file.endswith(".parquet"):
            
            # Extract site information from folder structure
            site = re.search(r"site=([^/]+)", root).group(1)
            year = re.search(r"year=([^/]+)", root).group(1)
            month = re.search(r"month=([^/]+)", root).group(1)
            
            if site!="LON":
                 break
            
            if year!=year_selected:
                break
            
            file_path = os.path.join(root, file)
            # Read the parquet file into a pandas DataFrame
            
            try:
                df = pd.read_parquet(file_path)
            except OSError:
                print(file + " is corrupted")
                continue
            
            print(file)
            
            # Filter by geometric altitude
            df = df[~df.GeometricAltitude.isnull()]
            df.GeometricAltitude = df.GeometricAltitude.astype(float)
            df = df[df.GeometricAltitude < maximum_altitude]
            
            df["Site"] = site
            df["Year"] = year
            df["Month"] = month
            
            # Get site latitude and longitude from `sites` DataFrame
            site_coords = sites[sites.site_abrev_name == site][["latitude", "longitude"]].values[0]
            site_lat, site_lon = site_coords
            
            try:
                # Calculate distance and filter rows within 50km
                df["Distance"] = haversine(df["Latitude"], df["Longitude"], site_lat, site_lon)
                df = df[df["Distance"] <= maximum_distance]
                
                # Drop the temporary 'Distance' column
                df.drop(columns=["Distance"], inplace=True)
            except TypeError:
                print(file + " has TypeError")
                continue
            
            if not df.empty:
                folder_dfs.append(df)

    if folder_dfs:
        print("concatening...")
        # Combine DataFrames from the current folder (if needed)
        df = pd.concat(folder_dfs, ignore_index=True)

        # Sort the dataframe by AircraftAddress, Callsign, and TimeRecPosition
        print("sorting values...")
        df.TimeRecPosition = pd.to_datetime(df.TimeRecPosition, format="ISO8601")
        df = df.sort_values(by=['AircraftAddress', 'Callsign', 'TimeRecPosition'])

        # Calculate the time difference within each group
        print("computing time diff...")
        df.loc[df.Callsign.isnull(), "Callsign"] = "NONE"
        df['time_diff'] = df.groupby(['AircraftAddress', 'Callsign'])['TimeRecPosition'].diff()

        # Initialize the journey column
        df['journey'] = 0

        # Create a flag for new journey based on time difference and new Callsign-AircraftAddress combination
        df['new_journey'] = (df['time_diff'] > time_threshold) | (df['Callsign'] != df['Callsign'].shift(1)) | \
                            (df['Site'] != df['Site'].shift(1)) | (df['AircraftAddress'] != df['AircraftAddress'].shift(1))

        # Use cumulative sum to assign journey IDs
        df['journey'] = df['new_journey'].cumsum() + 1

        # Calculate the time flown for each journey
        print("computing time flown...")
        df['timeFlown'] = df.groupby('journey')['TimeRecPosition'].transform(lambda x: x.max() - x.min())
        dfTime = df.drop_duplicates(subset=['journey']).reset_index(drop=True)
        final_table.append(dfTime.groupby(["Site", "Year", "Month"])["timeFlown"].sum().reset_index())
        print("")
        
final_table = [df for df in final_table if not df.empty]
final_concat = pd.concat(final_table, ignore_index=True)
final_by_site = pd.concat(final_table, ignore_index=True).groupby("Site").timeFlown.sum()
final_by_site


London_2024-07-01.parquet
London_2024-07-02.parquet
London_2024-07-03.parquet
London_2024-07-04.parquet
London_2024-07-05.parquet
London_2024-07-06.parquet
London_2024-07-07.parquet
London_2024-07-08.parquet
London_2024-07-09.parquet
London_2024-07-10.parquet
London_2024-07-11.parquet
London_2024-07-12.parquet
London_2024-07-13.parquet
London_2024-07-14.parquet
London_2024-07-15.parquet
London_2024-07-16.parquet
London_2024-07-17.parquet
London_2024-07-18.parquet
London_2024-07-19.parquet
London_2024-07-20.parquet
London_2024-07-21.parquet
London_2024-07-22.parquet
London_2024-07-23.parquet
London_2024-07-24.parquet
London_2024-07-25.parquet
London_2024-07-26.parquet
London_2024-07-27.parquet
London_2024-07-28.parquet
London_2024-07-29.parquet
London_2024-07-30.parquet
London_2024-07-31.parquet
concatening...
sorting values...
computing time diff...
computing time flown...

London_2024-08-01.parquet
London_2024-08-02.parquet
London_2024-08-03.parquet
London_2024-08-04.parquet
London_20

Site
LON   608 days 18:44:58.060000
Name: timeFlown, dtype: timedelta64[ns]

In [6]:
df.columns

Index(['EmitterCategory', 'GBS', 'ModeA', 'TimeRecPosition', 'AircraftAddress',
       'Latitude', 'Longitude', 'GeometricAltitude', 'FlightLevel',
       'BarometricVerticalRate', 'GeoVertRateExceeded',
       'GeometricVerticalRate', 'GroundSpeed', 'TrackAngle', 'Callsign',
       'AircraftStopped', 'GroundTrackValid', 'GroundHeadingProvided',
       'MagneticNorth', 'SurfaceGroundSpeed', 'SurfaceGroundTrack', 'Site',
       'Year', 'Month', 'time_diff', 'journey', 'new_journey', 'timeFlown'],
      dtype='object')

In [ ]:
pd.DataFrame(final_by_site)

,timeFlown
Site,
AUS,511 days 13:36:33.690000
BRE,71 days 09:03:34.600000
BRU,33 days 15:56:38.180000
BUC,171 days 07:16:19.710000
CYP,97 days 02:41:29.560000
LON,608 days 18:44:58.060000
LUX,96 days 21:16:41.450000
PAP,30 days 23:08:20.620000
SAR,1 days 08:11:02.950000
